# Credit Risk Modeling – Feature Engineering & Modeling

This notebook is part of an end-to-end credit risk modeling project completed
during my Data Science Internship at HDFC Capital Advisors Ltd.

⚠️ Note: Due to data confidentiality, raw datasets are not included.
The notebook demonstrates methodology, feature engineering logic,
modeling approach, and evaluation techniques.


In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_rows', 100000)

In [3]:
pd.set_option('display.max_columns', 140)

# Data Cleaning


## Accounts Data Train

In [4]:
LOAD_DATA = False  # Set to True only in secure local environment

if LOAD_DATA:
    with open('./data/senior_ds_test/data/train/accounts_data_train.json', 'r') as f:
        df = json.load(f)
else:
    df = []

flat_list = [item for sublist in df for item in sublist]
df_acc = pd.DataFrame(flat_list)

In [5]:
df_acc.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550
2,Credit card,80996.445,0.0,2020-06-29,NaN,000000000000000000,AAA10545297
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888
4,Credit card,10480.500,0.0,2014-09-10,NaN,0000000000000000000000000000000000000000000000...,AAA20326915


In [6]:
df_acc.columns

Index(['credit_type', 'loan_amount', 'amount_overdue', 'open_date',
       'closed_date', 'payment_hist_string', 'uid'],
      dtype='object')

In [7]:
df_acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245310 entries, 0 to 1245309
Data columns (total 7 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   credit_type          1245310 non-null  object 
 1   loan_amount          1245307 non-null  float64
 2   amount_overdue       1245310 non-null  float64
 3   open_date            1245310 non-null  object 
 4   closed_date          782275 non-null   object 
 5   payment_hist_string  1245310 non-null  object 
 6   uid                  1245310 non-null  object 
dtypes: float64(2), object(5)
memory usage: 66.5+ MB


In [8]:
df_acc.isnull().sum()

credit_type                 0
loan_amount                 3
amount_overdue              0
open_date                   0
closed_date            463035
payment_hist_string         0
uid                         0
dtype: int64

In [10]:
df_acc.nunique()

credit_type                15
loan_amount            189286
amount_overdue           1266
open_date                2923
closed_date              2913
payment_hist_string     45212
uid                    223918
dtype: int64

In [11]:
df_acc["credit_type"].value_counts()

credit_type
Consumer credit                                 908741
Credit card                                     292422
Car loan                                         20237
Mortgage                                         13196
Microloan                                         7766
Loan for business development                     1466
Another type of loan                               732
Unknown type of loan                               374
Loan for working capital replenishment             320
Real estate loan                                    17
Cash loan (non-earmarked)                           17
Loan for the purchase of equipment                  16
Loan for purchase of shares (margin lending)         4
Interbank credit                                     1
Mobile operator loan                                 1
Name: count, dtype: int64

In [12]:
df_acc["open_date"]=pd.to_datetime(df_acc["open_date"],format='%Y-%m-%d')
df_acc["closed_date"]=pd.to_datetime(df_acc["closed_date"],format='%Y-%m-%d')

In [13]:
df_acc[df_acc.duplicated(subset=["uid",'open_date','closed_date',"loan_amount","credit_type"])].sort_values(by='uid')

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
139,Credit card,0.000000e+00,0.000,2019-08-20,NaT,000000000000000000000000000000000000000000000000,AAD16503034
194,Car loan,1.449657e+06,0.000,2014-08-17,2019-05-01,0000000000000000000000000000000000000000000000...,AAD61169674
309,Credit card,0.000000e+00,0.000,2018-01-12,NaT,0000000000000000000000000000000000000000000000...,AAE71583603
311,Credit card,1.350000e+05,0.000,2019-03-24,NaT,0000000000000000000000000000000000000000000000...,AAE71583603
318,Credit card,1.350000e+05,0.000,2019-03-24,NaT,0000000000000000000000000000000000000000000000...,AAE71583603
638,Consumer credit,2.250000e+05,0.000,2013-10-08,2016-10-18,0000000000000000000000000000000000000000000000...,AAI20535173
664,Consumer credit,4.500000e+05,0.000,2017-12-04,NaT,0000000000000000000000000000000000000000000000...,AAI74342585
680,Credit card,1.800000e+05,0.000,2019-05-10,NaT,0000000000000000000000000000000000000000000000...,AAJ06918311
721,Credit card,2.250000e+05,0.000,2019-06-10,NaT,0000000000000000000000000000000000000000000000...,AAJ42627128
738,Consumer credit,7.290000e+05,0.000,2019-12-31,NaT,000000000000000018048078108000000000,AAJ61747027


limit_output extension: Maximum message size of 10000 exceeded with 1953012 characters

In [14]:
df_acc.duplicated(subset=["uid", "open_date", "closed_date","loan_amount","credit_type"]).sum()

7792

In [15]:
# Size of raw data
df_acc.shape

(1245310, 7)

In [16]:
#dropping duplicate rows
df_acc = df_acc.sort_values(by="uid").drop_duplicates(subset=["uid", "open_date", "closed_date","loan_amount"],keep='first')


In [17]:
#Size after duplicated rows are dropped. - 7792 duplicates dropped. 
df_acc.shape

(1237159, 7)

In [18]:
# checking if loan amount column has null values.

df_acc.loc[df_acc["loan_amount"].isnull()]

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
65605,Another type of loan,NaN,0.0,2021-01-01,2020-10-21,,BJV01354901
149202,Another type of loan,NaN,0.0,2021-01-01,NaT,,DCX71532515
613183,Another type of loan,NaN,0.0,2021-01-01,NaT,,MVG95017452


In [19]:
#dropping rows where loan amount has no value. - 3 rows

df_acc = df_acc.dropna(subset=['loan_amount'])

In [20]:
df_acc.shape

(1237156, 7)

In [21]:
# checking if loan amount has values that are 0. 

df_acc[df_acc['loan_amount']==0].head(40)

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
6,Credit card,0.0,0.0,2014-09-10,2018-08-30,0000000000000000000000000000000000000000000000...,AAA20326915
65,Credit card,0.0,0.0,2019-06-11,2020-04-30,000000000000000000000000000000,AAB67762691
89,Credit card,0.0,0.0,2018-06-27,NaT,0000000000000000000000000000000000000000000000...,AAC13959291
139,Credit card,0.0,0.0,2019-08-20,NaT,000000000000000000000000000000000000000000000000,AAD16503034
150,Credit card,0.0,0.0,2020-10-03,NaT,000000000,AAD30276592
159,Credit card,0.0,0.0,2020-02-13,NaT,000000000000000000000000000000,AAD49390925
165,Credit card,0.0,0.0,2017-05-12,2019-11-06,0000000000000000000000000000000000000000000000...,AAD51754962
200,Credit card,0.0,0.0,2018-11-21,2019-05-10,000000000000000,AAD66975621
237,Credit card,0.0,0.0,2013-10-20,2018-09-16,0000000000000000000000000000000000000280580881...,AAE20783392
236,Credit card,0.0,0.0,2013-10-20,2018-02-10,0000000000000000000000000000000000000000000000...,AAE20783392


limit_output extension: Maximum message size of 10000 exceeded with 10038 characters

In [22]:
(df_acc['loan_amount']==0).sum()

45773

In [23]:
#dropping rows where loan amount is 0. 

df_acc = df_acc[df_acc['loan_amount']!=0]

#45773 rows removed.

In [24]:
df_acc.shape

(1191383, 7)

In [25]:
#checking if closed date is earlier than the open date. 

df_acc[(df_acc['open_date'] > df_acc['closed_date'])]

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
91932,Consumer credit,153000.0,0.0,2019-07-08,2019-06-17,,BXW18347652
143642,Consumer credit,450000.0,0.0,2017-05-27,2017-04-14,,CZP89943956
147697,Consumer credit,360000.0,0.0,2013-06-05,2013-03-14,,DBX47652567
212896,Credit card,675000.0,0.0,2019-03-27,2015-11-29,,ELL49792531
380416,Consumer credit,711450.0,0.0,2018-03-17,2012-09-03,,HXU23242159
642848,Mortgage,2250000.0,0.0,2017-03-12,2013-10-18,,NLJ87108009
664086,Consumer credit,93564.0,0.0,2017-06-02,2016-12-02,,NWS78995671
752854,Credit card,225000.0,0.0,2014-08-06,1905-12-13,,PSI76661644
920889,Consumer credit,1350000.0,0.0,2018-07-29,2018-02-04,,TFK88667051
941309,Consumer credit,157500.0,0.0,2015-11-22,2015-10-09,,TQK82834352


In [26]:
df_acc[(df_acc['open_date'] > df_acc['closed_date'])].shape

(12, 7)

In [27]:
df_acc = df_acc[~(df_acc['open_date'] > df_acc['closed_date'])]


df_acc.shape
# drops 12 rows. 

(1191371, 7)

In [28]:
#Feature 1 - Duration of account() in months. 

df_acc['duration_in_months'] = (df_acc['closed_date'] - df_acc['open_date']).dt.days / 30

df_acc.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,duration_in_months
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550,17.266667
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550,16.800000
2,Credit card,80996.445,0.0,2020-06-29,NaT,000000000000000000,AAA10545297,NaN
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888,3.066667
11,Credit card,45000.000,0.0,2020-06-28,NaT,000000000000000000,AAA20326915,NaN


In [29]:
df_acc[(df_acc['duration_in_months'] > 1) & (df_acc['payment_hist_string'].isnull())]


,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,duration_in_months


In [30]:
df_acc[(df_acc['duration_in_months'] > 1) & (df_acc['payment_hist_string']=='')]

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,duration_in_months


In [31]:
df_acc[df_acc['open_date'].isnull()]

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,duration_in_months


In [32]:
#no. of open accounts. 

print("Number of open accounts:")
df_acc['closed_date'].isnull().sum()

Number of open accounts:


432927

In [33]:
df_acc.shape

(1191371, 8)

In [34]:
df_acc.columns

Index(['credit_type', 'loan_amount', 'amount_overdue', 'open_date',
       'closed_date', 'payment_hist_string', 'uid', 'duration_in_months'],
      dtype='object')

In [35]:
df_acc.loc[0]

credit_type                                              Consumer credit
loan_amount                                                     272745.0
amount_overdue                                                       0.0
open_date                                            2018-09-22 00:00:00
closed_date                                          2020-02-22 00:00:00
payment_hist_string    0000000000000000000000100000000000000000000000...
uid                                                          AAA09044550
duration_in_months                                             17.266667
Name: 0, dtype: object

### Accounts Features

##### DURATION 

In [36]:
duration_aggregates = df_acc.groupby('uid')['duration_in_months'].agg(['mean', 'sum', 'min', 'max']).reset_index()
duration_aggregates.columns = ['uid', 'mean_duration_months', 'total_duration_months', 'min_duration_months', 'max_duration_months']

# Merge the aggregates back into df_acc
df_acc = df_acc.merge(duration_aggregates, on='uid', how='left')

In [37]:
df_acc.shape

(1191371, 12)

##### TIME BASED

In [38]:
print(df_acc["loan_amount"].describe())
print()
print('Maximum:',df_acc["loan_amount"].max())
print('Minimum:',df_acc["loan_amount"].min())

count    1.191371e+06
mean     3.644118e+05
std      1.114348e+06
min      4.500000e-01
25%      5.850000e+04
50%      1.350000e+05
75%      3.155387e+05
max      3.960000e+08
Name: loan_amount, dtype: float64

Maximum: 396000000.0
Minimum: 0.45


In [39]:
earliest_acc = df_acc.groupby('uid')['open_date'].min().reset_index()
earliest_acc.columns = ['uid', 'earliest_acc_date']

# Extract year, month, and day from earliest_acc_date
earliest_acc['earliest_acc_year'] = earliest_acc['earliest_acc_date'].dt.year
earliest_acc['earliest_acc_month'] = earliest_acc['earliest_acc_date'].dt.month
earliest_acc['earliest_acc_day'] = earliest_acc['earliest_acc_date'].dt.day

# Merge the extracted columns back into df_acc
df_acc = df_acc.merge(earliest_acc[['uid', 'earliest_acc_year', 'earliest_acc_month', 'earliest_acc_day']], on='uid', how='left')

In [40]:
latest_acc = df_acc.groupby('uid')['open_date'].max().reset_index()
latest_acc.columns = ['uid', 'latest_acc_date']

# Extract year, month, and day from latest_acc_date
latest_acc['latest_acc_year'] = latest_acc['latest_acc_date'].dt.year
latest_acc['latest_acc_month'] = latest_acc['latest_acc_date'].dt.month
latest_acc['latest_acc_day'] = latest_acc['latest_acc_date'].dt.day

df_acc = df_acc.merge(latest_acc[['uid', 'latest_acc_year', 'latest_acc_month', 'latest_acc_day']], on='uid', how='left')

In [41]:
df_acc.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,duration_in_months,mean_duration_months,total_duration_months,min_duration_months,max_duration_months,earliest_acc_year,earliest_acc_month,earliest_acc_day,latest_acc_year,latest_acc_month,latest_acc_day
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550,17.266667,17.033333,34.066667,16.800000,17.266667,2018,3,8,2018,9,22
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550,16.800000,17.033333,34.066667,16.800000,17.266667,2018,3,8,2018,9,22
2,Credit card,80996.445,0.0,2020-06-29,NaT,000000000000000000,AAA10545297,NaN,NaN,0.000000,NaN,NaN,2020,6,29,2020,6,29
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888,3.066667,3.066667,3.066667,3.066667,3.066667,2020,6,9,2020,6,9
4,Credit card,45000.000,0.0,2020-06-28,NaT,000000000000000000,AAA20326915,NaN,4.058333,16.233333,1.033333,10.666667,2014,9,10,2020,6,28


In [42]:
df_acc.shape

(1191371, 18)

##### LOAN AMOUNT

In [43]:
#loan_amount_categories.
bins = [0, 100000, 500000, 1000000, 5000000, 10000000, 50000000,100000000,500000000]

labels = ['0-100k', '100k-500k', '500k-1M', '1M-5M', '5M-10M', '10M-50M','50M-100M','100M-500M']

df_acc['loan_amount_category'] = pd.cut(df_acc['loan_amount'], bins=bins, labels=labels)
df_acc.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,duration_in_months,mean_duration_months,total_duration_months,min_duration_months,max_duration_months,earliest_acc_year,earliest_acc_month,earliest_acc_day,latest_acc_year,latest_acc_month,latest_acc_day,loan_amount_category
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550,17.266667,17.033333,34.066667,16.800000,17.266667,2018,3,8,2018,9,22,100k-500k
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550,16.800000,17.033333,34.066667,16.800000,17.266667,2018,3,8,2018,9,22,0-100k
2,Credit card,80996.445,0.0,2020-06-29,NaT,000000000000000000,AAA10545297,NaN,NaN,0.000000,NaN,NaN,2020,6,29,2020,6,29,0-100k
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888,3.066667,3.066667,3.066667,3.066667,3.066667,2020,6,9,2020,6,9,0-100k
4,Credit card,45000.000,0.0,2020-06-28,NaT,000000000000000000,AAA20326915,NaN,4.058333,16.233333,1.033333,10.666667,2014,9,10,2020,6,28,0-100k


In [ ]:
#Total of each category.
df_acc["loan_amount_category"].value_counts()

In [44]:
# Metrics on Loan Amount
df_acc["avg_loan_amount"] = df_acc["uid"].map(df_acc.groupby("uid")["loan_amount"].mean())
df_acc["median_loan_amount"] = df_acc["uid"].map(df_acc.groupby("uid")["loan_amount"].median())
df_acc["max_loan_amt"] = df_acc["uid"].map(df_acc.groupby("uid")["loan_amount"].max())
df_acc["min_loan_amt"] = df_acc["uid"].map(df_acc.groupby("uid")["loan_amount"].min())

In [45]:
#avg. payment to be made each month. 
df_acc["avg_monthly_payment"] = (df_acc["loan_amount"]/df_acc["duration_in_months"]).round(2)

In [46]:
df_acc.shape

(1191371, 24)

In [47]:
df_acc.columns


Index(['credit_type', 'loan_amount', 'amount_overdue', 'open_date',
       'closed_date', 'payment_hist_string', 'uid', 'duration_in_months',
       'mean_duration_months', 'total_duration_months', 'min_duration_months',
       'max_duration_months', 'earliest_acc_year', 'earliest_acc_month',
       'earliest_acc_day', 'latest_acc_year', 'latest_acc_month',
       'latest_acc_day', 'loan_amount_category', 'avg_loan_amount',
       'median_loan_amount', 'max_loan_amt', 'min_loan_amt',
       'avg_monthly_payment'],
      dtype='object')

#### OVERDUES

In [48]:
# Metrics on Amount Overdue
df_acc["avg_overdues"] = df_acc["uid"].map(df_acc.groupby("uid")["amount_overdue"].mean())
df_acc["median_overdues"] = df_acc["uid"].map(df_acc.groupby("uid")["amount_overdue"].median())
df_acc["max_overdues"] = df_acc["uid"].map(df_acc.groupby("uid")["amount_overdue"].max())
df_acc["min_overdues"] = df_acc["uid"].map(df_acc.groupby("uid")["amount_overdue"].min())

In [49]:
df_acc.shape

(1191371, 28)

#### PAYMENT HISTORY

In [50]:
#Payment details
def analyze_payment_history(payment_history):
    months = [payment_history[i:i+3] for i in range(0, len(payment_history), 3)]

    late_payment_count = sum(1 for month in months if month != "000")
    max_consecutive_late_payments = 0
    consecutive_late_payments = 0
    last_payment_was_late = False
    
    for month in months:
        if month != "000":
            consecutive_late_payments = consecutive_late_payments + 1 if last_payment_was_late else 1
            last_payment_was_late = True
        else:
            max_consecutive_late_payments = max(max_consecutive_late_payments, consecutive_late_payments)
            consecutive_late_payments = 0
            last_payment_was_late = False
    
    max_consecutive_late_payments = max(max_consecutive_late_payments, consecutive_late_payments)
    on_time_payment_count = len(months) - late_payment_count
    time_since_last_late_payment = next((i for i, month in enumerate(reversed(months)) if month != "000"), len(months))

    return {
        'late_payments': late_payment_count,
        'on_time_payments': on_time_payment_count,
        'max_consecutive_late_payments': max_consecutive_late_payments,
        'time_since_last_late_payment': time_since_last_late_payment
    }
# Apply the function to the 'payment_hist_string' column and expand the result into separate columns
df_analysis = df_acc['payment_hist_string'].apply(analyze_payment_history).apply(pd.Series)

# Concatenate the results back to the original DataFrame
df_acc = pd.concat([df_acc, df_analysis], axis=1)

In [51]:
# metrics for on time payments 
df_acc["avg_ontime_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['on_time_payments'].mean().round(2))
df_acc["median_ontime_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['on_time_payments'].median())
df_acc["max_ontime_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['on_time_payments'].max())
df_acc["min_ontime_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['on_time_payments'].min())

In [52]:
# metrics for late payments 
df_acc["avg_late_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['late_payments'].mean().round(2))
df_acc["median_late_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['late_payments'].median())
df_acc["max_late_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['late_payments'].max())
df_acc["min_late_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['late_payments'].min())

In [53]:
# metrics for max_consecutive_late_payments
df_acc["avg_consecutive_late_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['max_consecutive_late_payments'].mean().round(2))
df_acc["median_consecutive_late_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['max_consecutive_late_payments'].median())
df_acc["max_consecutive_late_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['max_consecutive_late_payments'].max())
df_acc["min_consecutive_late_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['max_consecutive_late_payments'].min())

In [54]:
df_acc["max_consecutive_late_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['max_consecutive_late_payments'].max())

In [55]:
df_acc.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,duration_in_months,mean_duration_months,total_duration_months,min_duration_months,max_duration_months,earliest_acc_year,earliest_acc_month,earliest_acc_day,latest_acc_year,latest_acc_month,latest_acc_day,loan_amount_category,avg_loan_amount,median_loan_amount,max_loan_amt,min_loan_amt,avg_monthly_payment,avg_overdues,median_overdues,max_overdues,min_overdues,late_payments,on_time_payments,max_consecutive_late_payments,time_since_last_late_payment,avg_ontime_payments,median_ontime_payments,max_ontime_payments,min_ontime_payments,avg_late_payments,median_late_payments,max_late_payments,min_late_payments,avg_consecutive_late_payments,median_consecutive_late_payments,min_consecutive_late_payments
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550,17.266667,17.033333,34.066667,16.800000,17.266667,2018,3,8,2018,9,22,100k-500k,138622.500000,138622.500,272745.000,4500.000,15796.04,0.0,0.0,0.0,0.0,1,16,2,9,15.00,15.0,16,14,1.50,1.5,2,1,1.50,1.5,2
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550,16.800000,17.033333,34.066667,16.800000,17.266667,2018,3,8,2018,9,22,0-100k,138622.500000,138622.500,272745.000,4500.000,267.86,0.0,0.0,0.0,0.0,2,14,2,9,15.00,15.0,16,14,1.50,1.5,2,1,1.50,1.5,2
2,Credit card,80996.445,0.0,2020-06-29,NaT,000000000000000000,AAA10545297,NaN,NaN,0.000000,NaN,NaN,2020,6,29,2020,6,29,0-100k,80996.445000,80996.445,80996.445,80996.445,NaN,0.0,0.0,0.0,0.0,0,6,0,6,6.00,6.0,6,6,0.00,0.0,0,0,0.00,0.0,0
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888,3.066667,3.066667,3.066667,3.066667,3.066667,2020,6,9,2020,6,9,0-100k,43771.500000,43771.500,43771.500,43771.500,14273.32,0.0,0.0,0.0,0.0,0,3,0,3,3.00,3.0,3,3,0.00,0.0,0,0,0.00,0.0,0
4,Credit card,45000.000,0.0,2020-06-28,NaT,000000000000000000,AAA20326915,NaN,4.058333,16.233333,1.033333,10.666667,2014,9,10,2020,6,28,0-100k,84513.857143,46696.500,235800.000,10480.500,NaN,0.0,0.0,0.0,0.0,0,6,31,6,10.29,6.0,45,1,4.43,0.0,31,0,4.43,0.0,31


In [56]:
df_acc.shape

(1191371, 43)

#### DELINQUENCY

In [57]:
def calculate_delinquency_metrics(payment_history_string, delinquency_threshold=60):
    try:
        total_payments = len(payment_history_string) // 3
        if total_payments == 0:
            return None, None, None, None  # No payments to evaluate
        
        delinquent_payments = 0
        total_days_past_due = 0

        for i in range(0, len(payment_history_string), 3):
            payment_status = payment_history_string[i:i+3]
            days_past_due = int(payment_status)

            total_days_past_due += days_past_due

            if days_past_due > delinquency_threshold:
                delinquent_payments += 1

        delinquency_rate = delinquent_payments / total_payments
    except ZeroDivisionError:
        delinquency_rate = None
    except ValueError:  # if there's an invalid payment history format
        delinquency_rate = None
        total_payments = None
        delinquent_payments = None
        total_days_past_due = None

    delinquency_rate = delinquency_rate * 100 if delinquency_rate is not None else None
    
    return delinquency_rate, total_payments, delinquent_payments, total_days_past_due

# Apply the function to the 'payment_hist_string' column
df_acc[['delinquency_rate', 'total_payments', 'delinquent_payments', 'total_DPD']] = df_acc['payment_hist_string'].apply(
    lambda x: pd.Series(calculate_delinquency_metrics(x))
)



In [58]:
# metrics for Total Payments
df_acc["avg_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['total_payments'].mean().round(2))
df_acc["median_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['total_payments'].median())
df_acc["max_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['total_payments'].max())
df_acc["min_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['total_payments'].min())

In [59]:
# metrics for Delinquent Payments
df_acc["avg_del_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['delinquent_payments'].mean().round(2))
df_acc["median_del_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['delinquent_payments'].median())
df_acc["max_del_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['delinquent_payments'].max())
df_acc["min_del_payments"] = df_acc["uid"].map(df_acc.groupby('uid')['delinquent_payments'].min())

In [60]:
# metrics for Days Past Due
df_acc["avg_DPD"] = df_acc["uid"].map(df_acc.groupby('uid')['total_DPD'].mean().round(2))
df_acc["median_DPD"] = df_acc["uid"].map(df_acc.groupby('uid')['total_DPD'].median())
df_acc["max_DPD"] = df_acc["uid"].map(df_acc.groupby('uid')['total_DPD'].max())
df_acc["min_DPD"] = df_acc["uid"].map(df_acc.groupby('uid')['total_DPD'].min())

In [61]:
# metrics for Deliquency Rates
df_acc["avg_DR"] = df_acc["uid"].map(df_acc.groupby('uid')['delinquency_rate'].mean().round(2))
df_acc["median_DR"] = df_acc["uid"].map(df_acc.groupby('uid')['delinquency_rate'].median())
df_acc["max_DR"] = df_acc["uid"].map(df_acc.groupby('uid')['delinquency_rate'].max())
df_acc["min_DR"] = df_acc["uid"].map(df_acc.groupby('uid')['delinquency_rate'].min())

In [62]:
df_acc.shape

(1191371, 63)

In [63]:
df_acc.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,duration_in_months,mean_duration_months,total_duration_months,min_duration_months,max_duration_months,earliest_acc_year,earliest_acc_month,earliest_acc_day,latest_acc_year,latest_acc_month,latest_acc_day,loan_amount_category,avg_loan_amount,median_loan_amount,max_loan_amt,min_loan_amt,avg_monthly_payment,avg_overdues,median_overdues,max_overdues,min_overdues,late_payments,on_time_payments,max_consecutive_late_payments,time_since_last_late_payment,avg_ontime_payments,median_ontime_payments,max_ontime_payments,min_ontime_payments,avg_late_payments,median_late_payments,max_late_payments,min_late_payments,avg_consecutive_late_payments,median_consecutive_late_payments,min_consecutive_late_payments,delinquency_rate,total_payments,delinquent_payments,total_DPD,avg_payments,median_payments,max_payments,min_payments,avg_del_payments,median_del_payments,max_del_payments,min_del_payments,avg_DPD,median_DPD,max_DPD,min_DPD,avg_DR,median_DR,max_DR,min_DR
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550,17.266667,17.033333,34.066667,16.800000,17.266667,2018,3,8,2018,9,22,100k-500k,138622.500000,138622.500,272745.000,4500.000,15796.04,0.0,0.0,0.0,0.0,1,16,2,9,15.00,15.0,16,14,1.50,1.5,2,1,1.50,1.5,2,0.0,17.0,0.0,10.0,16.50,16.5,17.0,16.0,0.00,0.0,0.0,0.0,34.00,34.0,58.0,10.0,0.00,0.0,0.000000,0.0
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550,16.800000,17.033333,34.066667,16.800000,17.266667,2018,3,8,2018,9,22,0-100k,138622.500000,138622.500,272745.000,4500.000,267.86,0.0,0.0,0.0,0.0,2,14,2,9,15.00,15.0,16,14,1.50,1.5,2,1,1.50,1.5,2,0.0,16.0,0.0,58.0,16.50,16.5,17.0,16.0,0.00,0.0,0.0,0.0,34.00,34.0,58.0,10.0,0.00,0.0,0.000000,0.0
2,Credit card,80996.445,0.0,2020-06-29,NaT,000000000000000000,AAA10545297,NaN,NaN,0.000000,NaN,NaN,2020,6,29,2020,6,29,0-100k,80996.445000,80996.445,80996.445,80996.445,NaN,0.0,0.0,0.0,0.0,0,6,0,6,6.00,6.0,6,6,0.00,0.0,0,0,0.00,0.0,0,0.0,6.0,0.0,0.0,6.00,6.0,6.0,6.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.000000,0.0
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888,3.066667,3.066667,3.066667,3.066667,3.066667,2020,6,9,2020,6,9,0-100k,43771.500000,43771.500,43771.500,43771.500,14273.32,0.0,0.0,0.0,0.0,0,3,0,3,3.00,3.0,3,3,0.00,0.0,0,0,0.00,0.0,0,0.0,3.0,0.0,0.0,3.00,3.0,3.0,3.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.000000,0.0
4,Credit card,45000.000,0.0,2020-06-28,NaT,000000000000000000,AAA20326915,NaN,4.058333,16.233333,1.033333,10.666667,2014,9,10,2020,6,28,0-100k,84513.857143,46696.500,235800.000,10480.500,NaN,0.0,0.0,0.0,0.0,0,6,31,6,10.29,6.0,45,1,4.43,0.0,31,0,4.43,0.0,31,0.0,6.0,0.0,0.0,14.71,6.0,76.0,1.0,4.14,0.0,29.0,0.0,2068.14,0.0,14477.0,0.0,5.45,0.0,38.157895,0.0


#### MONTHLY SEGGREGATION OF PAYMENT HISTORY

In [64]:
# splitting payment history string acc to last 3 mons, 6 mons, and 9 mons. 
def split_payment_history(payment_hist_string):
    last_1_month = None
    last_3_months = None
    last_6_months = None
    last_9_months = None
    if len(payment_hist_string) >= 3:
        last_1_month = payment_hist_string[:3]
    if len(payment_hist_string) >= 3*3:
        last_3_months = payment_hist_string[:3*3]
    if len(payment_hist_string) >= 6*3:
        last_6_months = payment_hist_string[:6*3]
    if len(payment_hist_string) >= 9*3:
        last_9_months = payment_hist_string[:9*3]
    
    return pd.Series([last_1_month,last_3_months, last_6_months, last_9_months])

# Apply the function to the 'payment_hist_string' column
df_acc[["last_1_month",'last_3_months', 'last_6_months', 'last_9_months']] = df_acc['payment_hist_string'].apply(split_payment_history)


##### PAST 1 MONTH

In [65]:
#avg delinquency in latest month.
def past_month_delinquency(last_1_month):
    if last_1_month and last_1_month.isdigit():
        return int(last_1_month)
    return 0

df_acc['DPD_last_1_month'] = df_acc['last_1_month'].apply(past_month_delinquency)

In [67]:
#deliquency over the past month
df_acc["avg_DPD_last_1_month"] = df_acc["uid"].map(df_acc.groupby('uid')['DPD_last_1_month'].mean().round(2))
df_acc["median_DPD_last_1_month"] = df_acc["uid"].map(df_acc.groupby('uid')['DPD_last_1_month'].median())
df_acc["max_DPD_last_1_month"] = df_acc["uid"].map(df_acc.groupby('uid')['DPD_last_1_month'].max())
df_acc["min_DPD_last_1_month"] = df_acc["uid"].map(df_acc.groupby('uid')['DPD_last_1_month'].min())

In [68]:
df_acc.shape

(1191371, 72)

##### PAST 3,6,9 MONTHS

In [69]:
#delinquency for last,3,6 and 9 months.

def delinquency(last_x_month):
    if pd.isna(last_x_month):
        return 0
    total_days = 0
    for i in range(0, len(last_x_month), 3):
        total_days += int(last_x_month[i:i+3])
    return total_days


df_acc["total_delinquency_3_mons"] = df_acc["last_3_months"].apply(delinquency)
df_acc["avg_DPD_3_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_3_mons'].mean().round(2))
df_acc["median_DPD_3_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_3_mons'].median())
df_acc["max_DPD_3_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_3_mons'].max())
df_acc["min_DPD_3_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_3_mons'].min())


df_acc["total_delinquency_6_mons"] = df_acc["last_6_months"].apply(delinquency)
df_acc["avg_DPD_6_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_6_mons'].mean().round(2))
df_acc["median_DPD_6_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_6_mons'].median())
df_acc["max_DPD_6_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_6_mons'].max())
df_acc["min_DPD_6_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_6_mons'].min())

df_acc["total_delinquency_9_mons"] = df_acc["last_9_months"].apply(delinquency)
df_acc["avg_DPD_9_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_9_mons'].mean().round(2))
df_acc["max_DPD_9_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_9_mons'].max())
df_acc["median_DPD_9_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_9_mons'].median())
df_acc["min_DPD_9_mons"] = df_acc["uid"].map(df_acc.groupby('uid')['total_delinquency_9_mons'].min())

In [ ]:
print(df_acc["delinquency_rate"].describe())


print()
print("Max. Delinquency: ",df_acc["delinquency_rate"].max())
print("Min. Delinquency: ",df_acc["delinquency_rate"].min())

In [ ]:
df_acc[df_acc["amount_overdue"]!=0].head(40)

In [ ]:
df_non_zero_dpd = df_acc.query("DPD_last_1_month != 0")

In [70]:
df_acc.shape

(1191371, 87)

#### CREDIT TYPES

In [71]:
pivot_mean = pd.pivot_table(df_acc, 
                            index='uid', 
                            columns='credit_type', 
                            values='loan_amount', 
                            aggfunc='mean', 
                            fill_value=0)

# Flatten the columns of the pivot table
pivot_mean.columns = [f'{credit}_mean_loan' for credit in pivot_mean.columns]

# Reset the index to merge with the original dataframe
pivot_mean.reset_index(inplace=True)

df_acc = df_acc.merge(pivot_mean, on='uid', how='left')

In [72]:
df_acc.shape

(1191371, 102)

In [73]:
pivot_median = pd.pivot_table(df_acc, 
                            index='uid', 
                            columns='credit_type', 
                            values='loan_amount', 
                            aggfunc='median', 
                            fill_value=0)

# Flatten the columns of the pivot table
pivot_median.columns = [f'{credit}_median_loan' for credit in pivot_median.columns]

# Reset the index to merge with the original dataframe
pivot_median.reset_index(inplace=True)

df_acc = df_acc.merge(pivot_median, on='uid', how='left')

In [74]:
df_acc.shape

(1191371, 117)

In [75]:
pivot_sum = pd.pivot_table(df_acc, 
                            index='uid', 
                            columns='credit_type', 
                            values='loan_amount', 
                            aggfunc='sum', 
                            fill_value=0)

# Flatten the columns of the pivot table
pivot_sum.columns = [f'{credit}_total_loan' for credit in pivot_sum.columns]

# Reset the index to merge with the original dataframe
pivot_sum.reset_index(inplace=True)

df_acc = df_acc.merge(pivot_sum, on='uid', how='left')

In [197]:
df_acc.head()

credit_type  loan_amount  amount_overdue  open_date closed_date  \
0  Consumer credit   272745.000             0.0 2018-09-22  2020-02-22   
1  Consumer credit     4500.000             0.0 2018-03-08  2019-07-25   
2      Credit card    80996.445             0.0 2020-06-29         NaT   
3  Consumer credit    43771.500             0.0 2020-06-09  2020-09-09   
4      Credit card    45000.000             0.0 2020-06-28         NaT   

                                 payment_hist_string          uid  \
0  0000000000000000000000100000000000000000000000...  AAA09044550   
1   000000000000000014044000000000000000000000000000  AAA09044550   
2                                 000000000000000000  AAA10545297   
3                                          000000000  AAA14112888   
4                                 000000000000000000  AAA20326915   

   duration_in_months  mean_duration_months  total_duration_months  \
0           17.266667             17.033333              34.066667   
1           16.800000             17.033333              34.066667   
2                 NaN                   NaN               0.000000   
3            3.066667              3.066667               3.066667   
4                 NaN             12.913333              64.566667   

   min_duration_months  max_duration_months  earliest_acc_year  \
0            16.800000            17.266667               2018   
1            16.800000            17.266667               2018   
2                  NaN                  NaN               2020   
3             3.066667             3.066667               2020   
4             1.033333            48.333333               2014   

   earliest_acc_month  earliest_acc_day  latest_acc_year  latest_acc_month  \
0                   3                 8             2018                 9   
1                   3                 8             2018                 9   
2                   6                29             2020                 6   
3                   6                 9             2020                 6   
4                   9                10             2020                 6   

   latest_acc_day loan_amount_category  avg_loan_amount  median_loan_amount  \
0              22            100k-500k       138622.500          138622.500   
1              22               0-100k       138622.500          138622.500   
2              29               0-100k        80996.445           80996.445   
3               9               0-100k        43771.500           43771.500   
4              28               0-100k        73949.625           46658.250   

   max_loan_amt  min_loan_amt  avg_monthly_payment  avg_overdues  \
0    272745.000      4500.000             15796.04           0.0   
1    272745.000      4500.000               267.86           0.0   
2     80996.445     80996.445                  NaN           0.0   
3     43771.500     43771.500             14273.32           0.0   
4    235800.000         0.000                  NaN           0.0   

   median_overdues  max_overdues  min_overdues  late_payments  \
0              0.0           0.0           0.0              1   
1              0.0           0.0           0.0              2   
2              0.0           0.0           0.0              0   
3              0.0           0.0           0.0              0   
4              0.0           0.0           0.0              0   

   on_time_payments  max_consecutive_late_payments  \
0                16                              2   
1                14                              2   
2                 6                              0   
3                 3                              0   
4                 6                             31   

   time_since_last_late_payment  avg_ontime_payments  median_ontime_payments  \
0                             9                 15.0                    15.0   
1                             9                 15.0                    15.0   
2                   

limit_output extension: Maximum message size of 10000 exceeded with 14359 characters

In [76]:
df_acc.shape

(1191371, 132)

#### LOAN STATUS

In [77]:
#Loan Status
df_acc['loan_status'] = df_acc['closed_date'].apply(lambda x: 'Open' if pd.isnull(x) else 'Closed')

In [78]:
df_acc.head()

credit_type  loan_amount  amount_overdue  open_date closed_date  \
0  Consumer credit   272745.000             0.0 2018-09-22  2020-02-22   
1  Consumer credit     4500.000             0.0 2018-03-08  2019-07-25   
2      Credit card    80996.445             0.0 2020-06-29         NaT   
3  Consumer credit    43771.500             0.0 2020-06-09  2020-09-09   
4      Credit card    45000.000             0.0 2020-06-28         NaT   

                                 payment_hist_string          uid  \
0  0000000000000000000000100000000000000000000000...  AAA09044550   
1   000000000000000014044000000000000000000000000000  AAA09044550   
2                                 000000000000000000  AAA10545297   
3                                          000000000  AAA14112888   
4                                 000000000000000000  AAA20326915   

   duration_in_months  mean_duration_months  total_duration_months  \
0           17.266667             17.033333              34.066667   
1           16.800000             17.033333              34.066667   
2                 NaN                   NaN               0.000000   
3            3.066667              3.066667               3.066667   
4                 NaN              4.058333              16.233333   

   min_duration_months  max_duration_months  earliest_acc_year  \
0            16.800000            17.266667               2018   
1            16.800000            17.266667               2018   
2                  NaN                  NaN               2020   
3             3.066667             3.066667               2020   
4             1.033333            10.666667               2014   

   earliest_acc_month  earliest_acc_day  latest_acc_year  latest_acc_month  \
0                   3                 8             2018                 9   
1                   3                 8             2018                 9   
2                   6                29             2020                 6   
3                   6                 9             2020                 6   
4                   9                10             2020                 6   

   latest_acc_day loan_amount_category  avg_loan_amount  median_loan_amount  \
0              22            100k-500k    138622.500000          138622.500   
1              22               0-100k    138622.500000          138622.500   
2              29               0-100k     80996.445000           80996.445   
3               9               0-100k     43771.500000           43771.500   
4              28               0-100k     84513.857143           46696.500   

   max_loan_amt  min_loan_amt  avg_monthly_payment  avg_overdues  \
0    272745.000      4500.000             15796.04           0.0   
1    272745.000      4500.000               267.86           0.0   
2     80996.445     80996.445                  NaN           0.0   
3     43771.500     43771.500             14273.32           0.0   
4    235800.000     10480.500                  NaN           0.0   

   median_overdues  max_overdues  min_overdues  late_payments  \
0              0.0           0.0           0.0              1   
1              0.0           0.0           0.0              2   
2              0.0           0.0           0.0              0   
3              0.0           0.0           0.0              0   
4              0.0           0.0           0.0              0   

   on_time_payments  max_consecutive_late_payments  \
0                16                              2   
1                14                              2   
2                 6                              0   
3                 3                              0   
4                 6                             31   

   time_since_last_late_payment  avg_ontime_payments  median_ontime_payments  \
0                             9                15.00                    15.0   
1                             9                15.00                    15.0   
2                   

limit_output extension: Maximum message size of 10000 exceeded with 20486 characters

In [79]:
df_acc.shape

(1191371, 133)

In [80]:
most_frequent_credit_type = df_acc.groupby('uid')['credit_type'].agg(lambda x: x.value_counts().idxmax())

df_acc['most_frequent_credit_type'] = df_acc['uid'].map(most_frequent_credit_type)

In [ ]:
df_acc.columns

### Feature Selection

In [81]:
columns_to_drop = [
    "credit_type",
    "loan_amount",
    "amount_overdue",
    "open_date",
    "closed_date",
    "payment_hist_string",
    "duration_in_months",
    "late_payments",
    "on_time_payments",
    "time_since_last_late_payment",
    "max_consecutive_late_payments",
    "delinquency_rate",
    "total_payments",
    "delinquent_payments",
    "total_DPD",
    "loan_amount_category",
    "median_ontime_payments",
    "median_late_payments",
    "median_consecutive_late_payments",
    "median_DR","max_DR","min_DR",
    "last_1_month",
    "last_3_months",
    "last_6_months",
    "last_9_months",
    "avg_monthly_payment",
    "total_delinquency_3_mons",
    "total_delinquency_6_mons",
    "total_delinquency_9_mons",
    "DPD_last_1_month",
    "median_DPD_last_1_month",
    "max_DPD_last_1_month",
    "min_DPD_last_1_month",
    "median_DPD_3_mons",
    "max_DPD_3_mons",
    "min_DPD_3_mons",
    "median_DPD_6_mons",
    "max_DPD_6_mons",
    "min_DPD_6_mons",
    "median_DPD_9_mons",
    "max_DPD_9_mons",
    "min_DPD_9_mons",
    "median_overdues"
    
]


df_acc = df_acc.drop(columns=columns_to_drop)

In [82]:
df_acc.shape

(1191371, 90)

In [83]:
df_acc["uid"].nunique()

222990

In [84]:
df_acc.duplicated(subset="uid")

0          False
1           True
2          False
3          False
4          False
           ...  
1191366     True
1191367     True
1191368     True
1191369     True
1191370     True
Length: 1191371, dtype: bool

In [85]:
df_acc = df_acc.drop_duplicates(subset="uid",keep="first")

In [86]:
df_acc.head()

uid  mean_duration_months  total_duration_months  \
0   AAA09044550             17.033333              34.066667   
2   AAA10545297                   NaN               0.000000   
3   AAA14112888              3.066667               3.066667   
4   AAA20326915              4.058333              16.233333   
11  AAA31604840             15.450000              30.900000   

    min_duration_months  max_duration_months  earliest_acc_year  \
0             16.800000            17.266667               2018   
2                   NaN                  NaN               2020   
3              3.066667             3.066667               2020   
4              1.033333            10.666667               2014   
11            12.600000            18.300000               2017   

    earliest_acc_month  earliest_acc_day  latest_acc_year  latest_acc_month  \
0                    3                 8             2018                 9   
2                    6                29             2020                 6   
3                    6                 9             2020                 6   
4                    9                10             2020                 6   
11                   2                15             2020                10   

    latest_acc_day  avg_loan_amount  median_loan_amount  max_loan_amt  \
0               22    138622.500000          138622.500    272745.000   
2               29     80996.445000           80996.445     80996.445   
3                9     43771.500000           43771.500     43771.500   
4               28     84513.857143           46696.500    235800.000   
11              10    318392.100000          304965.000    687150.000   

    min_loan_amt  avg_overdues  max_overdues  min_overdues  \
0       4500.000           0.0           0.0           0.0   
2      80996.445           0.0           0.0           0.0   
3      43771.500           0.0           0.0           0.0   
4      10480.500           0.0           0.0           0.0   
11     41845.500           0.0           0.0           0.0   

    avg_ontime_payments  max_ontime_payments  min_ontime_payments  \
0                 15.00                   16                   14   
2                  6.00                    6                    6   
3                  3.00                    3                    3   
4                 10.29                   45                    1   
11                15.60                   43                    2   

    avg_late_payments  max_late_payments  min_late_payments  \
0                1.50                  2                  1   
2                0.00                  0                  0   
3                0.00                  0                  0   
4                4.43                 31                  0   
11               0.00                  0                  0   

    avg_consecutive_late_payments  min_consecutive_late_payments  \
0                            1.50                              2   
2                            0.00                              0   
3                            0.00                              0   
4                            4.43                             31   
11                           0.00                              0   

    avg_payments  median_payments  max_payments  min_payments  \
0          16.50             16.5          17.0          16.0   
2           6.00              6.0           6.0           6.0   
3           3.00              3.0           3.0           3.0   
4          14.71              6.0          76.0           1.0   
11         15.60             12.0          43.0           2.0   

    avg_del_payments  median_del_payments  max_del_payments  min_del_payments  \
0               0.00                  0.0               0.0               0.0   
2               0.00                  0.0               0.0               0.0   
3               0.00                  0.0               0.0               0.0   
4               4.1

limit_output extension: Maximum message size of 10000 exceeded with 15332 characters

In [87]:
acc_prefix = 'accList_'
df_acc = df_acc.rename(columns=lambda x: acc_prefix + x if x != 'uid' else x)

In [88]:
user_dict = df_acc[df_acc["uid"]=='AAA09044550'].to_dict('records')

In [89]:
user_dict

[{'uid': 'AAA09044550',
  'accList_mean_duration_months': 17.03333333333333,
  'accList_total_duration_months': 34.06666666666666,
  'accList_min_duration_months': 16.8,
  'accList_max_duration_months': 17.266666666666666,
  'accList_earliest_acc_year': 2018,
  'accList_earliest_acc_month': 3,
  'accList_earliest_acc_day': 8,
  'accList_latest_acc_year': 2018,
  'accList_latest_acc_month': 9,
  'accList_latest_acc_day': 22,
  'accList_avg_loan_amount': 138622.5,
  'accList_median_loan_amount': 138622.5,
  'accList_max_loan_amt': 272745.0,
  'accList_min_loan_amt': 4500.0,
  'accList_avg_overdues': 0.0,
  'accList_max_overdues': 0.0,
  'accList_min_overdues': 0.0,
  'accList_avg_ontime_payments': 15.0,
  'accList_max_ontime_payments': 16,
  'accList_min_ontime_payments': 14,
  'accList_avg_late_payments': 1.5,
  'accList_max_late_payments': 2,
  'accList_min_late_payments': 1,
  'accList_avg_consecutive_late_payments': 1.5,
  'accList_min_consecutive_late_payments': 2,
  'accList_avg_pa

In [90]:
df_acc.to_csv('df_acc.csv', index=False)

# EDA


In [106]:
df_acc.columns

Index(['uid', 'mean_duration_months', 'total_duration_months',
       'min_duration_months', 'max_duration_months', 'earliest_acc_year',
       'earliest_acc_month', 'earliest_acc_day', 'latest_acc_year',
       'latest_acc_month', 'latest_acc_day', 'avg_loan_amount',
       'median_loan_amount', 'max_loan_amt', 'min_loan_amt', 'avg_overdues',
       'max_overdues', 'min_overdues', 'avg_payments', 'median_payments',
       'max_payments', 'min_payments', 'avg_del_payments',
       'median_del_payments', 'max_del_payments', 'min_del_payments',
       'avg_DPD', 'median_DPD', 'max_DPD', 'min_DPD', 'avg_DR',
       'avg_DPD_last_1_month', 'avg_DPD_3_mons', 'avg_DPD_6_mons',
       'avg_DPD_9_mons', 'Another type of loan_mean_loan',
       'Car loan_mean_loan', 'Cash loan (non-earmarked)_mean_loan',
       'Consumer credit_mean_loan', 'Credit card_mean_loan',
       'Interbank credit_mean_loan', 'Loan for business development_mean_loan',
       'Loan for purchase of shares (margin lending)

## Credit Type

In [ ]:
credit_type_counts = df_acc["credit_type"].value_counts().head(5)
plt.figure(figsize=(6, 5))
plt.bar(credit_type_counts.index, credit_type_counts.values,width=0.3)
plt.xticks(rotation=45)
plt.xlabel('Credit Type')
plt.ylabel('Count')
plt.title('Count of Entries for Each Credit Type')
plt.show()

Most popular type of loan/credit is 'Consumer Credit'

In [ ]:
filt = df_acc[df_acc["amount_overdue"]!=0]

# Box plot for amount overdue by credit type
plt.figure(figsize=(12, 6))
sns.boxplot(x='credit_type', y='amount_overdue', data=filt, palette='pastel')

plt.xlabel('Credit Type')
plt.xticks(rotation=45)
plt.ylabel('Amount Overdue')
plt.title('Amount Overdue by Credit Type')

plt.show()

In [ ]:
df_acc.sort_values(by="amount_overdue",ascending=False).head(20)

**Most of the overdue amount for consumer credit and credit card is concentrated between the range of 0 to 50000000.**

In [ ]:
# Delinquency Rate
filt=df_acc[df_acc["delinquency_rate"]!=0]

plt.figure(figsize=(15, 5))
sns.scatterplot(x='credit_type', y='delinquency_rate', data=filt,alpha=0.3)
plt.title('Delinquency Rate by Credit Type')
plt.xlabel('Credit Type')
plt.ylabel('Delinquency Rate')
plt.xticks(rotation=45)
plt.show()

## Delinquency Rate

In [ ]:
avg_delinquency = df_acc.groupby("credit_type")["delinquency_rate"].mean().reset_index().sort_values('delinquency_rate',ascending=False).head(10)


In [ ]:
plt.figure(figsize=(20,5))
plt.bar(avg_delinquency["credit_type"], avg_delinquency["delinquency_rate"], color='pink',width=0.4)
plt.xticks(rotation=45)
plt.xlabel("Credit Type")
plt.ylabel("Average Delinquency")
plt.title("Delinquency Rate across Credit Types")
plt.grid(True)
plt.show()

Maximum Deliquency is shown by customers who have taken a cash loan.

In [ ]:
df_acc['open_year'] = df_acc["open_date"].dt.year

avg_delinquency = df_acc.groupby("open_year")["delinquency_rate"].mean()
plt.plot(avg_delinquency.index, avg_delinquency.values, marker='o', linestyle='-', color='green')
plt.xlabel('Open Date')
plt.ylabel('Delinquency Rate')
plt.title('Delinquency Rate Over Time')
plt.show()


**Highest delinquency is potraye by customers who have opened accounts in 2013.**

**There is a sudden drop in delinquency rate between 2019 to 2020.**

**Also Delinquecy decrease as the opening date near the present time.**

In [ ]:
monthly_delinquency = df_acc.groupby([df_acc['open_year'], df_acc['month_name']])['delinquency_rate'].mean().reset_index()

print(monthly_delinquency.max())


In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(monthly_delinquency['month_name'], monthly_delinquency['delinquency_rate'])
plt.xlabel('Month')
plt.ylabel('Delinquency Rate (%)')
plt.title('Monthly Delinquency Rate over Time')
plt.xticks(rotation=45)
plt.yticks([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
plt.grid(True)
plt.show()

**Highest delinquency are potrayed by customers who open accounts in the month of October.**

In [ ]:
avg_late_payments = df_acc.groupby('late_payments')['delinquency_rate'].mean()

plt.figure(figsize=(10, 6))
plt.plot(avg_late_payments.index, avg_late_payments.values) 
plt.xlabel("Average Number of Late Payments")
plt.ylabel("Delinquency Rate")
plt.title("Delinquency Rate vs. Average Late Payments")
plt.grid(True)
plt.show()

## Date

In [ ]:
a = df_acc.groupby(df_acc['open_date'].dt.month_name())['uid'].nunique().sort_values()
b = df_acc.groupby(df_acc['closed_date'].dt.month_name())['uid'].nunique().sort_values()


plt.subplot(1,2,1)
a.plot(kind='bar', label='Opened Accounts',color='r',alpha=0.3)
plt.xlabel('Month')
plt.ylabel('Number of Accounts')
plt.title('Opened Loan Accounts by Month')

plt.subplot(1,2,2)
b.plot(kind='bar', label='Closed Accounts',color='b',alpha=0.3)
plt.xlabel('Month')
plt.ylabel('Number of Accounts')
plt.title('Closed Loan Accounts by Month')

plt.tight_layout()
plt.show()

In [ ]:
plt.bar(['On-Time Payments', 'Late Payments'], [df_acc['on_time_payments'].sum(), df_acc['late_payments'].sum()])
plt.xlabel('Payment Type')
plt.ylabel('Total Count')
plt.title('On-Time vs. Late Payments (Total Count)')
plt.show()

**Late payments are significantly lower than on time payments.**

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_acc['time_since_last_late_payment'], kde=True)
plt.title('Distribution of Time Since Last Late Payment')
plt.xlabel('Time Since Last Late Payment (days)')
plt.ylabel('Frequency')
plt.show()

**Right Skewed distribution suggests that,, majority of late payments were made recently i.e. less than 20 days.**

## Day

In [ ]:
# Box plot for loan amount by day opened
plt.figure(figsize=(14, 6))
sns.boxplot(x='day_opened', y='loan_amount', data=df_acc, palette='coolwarm')

# Adding labels and title
plt.xlabel('Day Opened')
plt.ylabel('Loan Amount')
plt.title('Loan Amount Distribution by Day Opened')

# Display the plot
plt.show()

**Majority of loans taken lie between 0 - 100000000.. very few payments go past 1CR. They have no uniqueness wrt to the days of the week**

## Loan Amount Category

In [ ]:
#understanding distribution of different categories. 
category = df_acc["loan_amount_category"].value_counts()
sns.countplot(x='loan_amount_category', data=df_acc, palette='coolwarm')
plt.xlabel('Loan Amount Category')
plt.ylabel('Count')
plt.title('Distribution of Loan Amounts by Category')
plt.xticks(rotation=45)
plt.show()

Maximum loans range btw 100k to 500k. 